## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

import math

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


__What variable(s) are the target(s) for your model?__ 'IS_SUCCESSFUL'

__What variable(s) are the feature(s) for your model?__ All other columns

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Display the data types and non-null count
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [5]:
# Convert STATUS to Y/N instead of 1/0.
application_df['STATUS'] = application_df['STATUS'].map({
    0: 'N',
    1: 'Y'
})

# Confirm changes
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  object
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 2.6+ MB


### Application Type

In [6]:
# Look at APPLICATION_TYPE value counts for binning
app_type = application_df['APPLICATION_TYPE'].value_counts()
app_type

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [7]:
# Define a cutoff value using percentiles
app_cutoff = app_type.quantile(0.25)

# Round the value to the nearest tens
app_cutoff_rounded = math.ceil(app_cutoff / 10) * 10
app_cutoff_rounded

10

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = app_type.loc[app_type.values < app_cutoff_rounded].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: count, dtype: int64

### Classification

In [9]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts.loc[class_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [11]:
# Define a cutoff value using percentiles
class_cutoff = class_counts.quantile(0.25)

# Round the value to the nearest tens
class_cutoff_rounded = math.ceil(app_cutoff / 10) * 10
class_cutoff_rounded

10

In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = class_counts.loc[class_counts.values < class_cutoff_rounded].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Other       98
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
Name: count, dtype: int64

### Feature Engineering

In [13]:
application_df.loc[application_df['INCOME_AMT'] != '0']['INCOME_AMT'].value_counts()

INCOME_AMT
25000-99999      3747
100000-499999    3374
1M-5M             955
1-9999            728
10000-24999       543
10M-50M           240
5M-10M            185
50M+              139
Name: count, dtype: int64

In [14]:
# Split the bounded values by the hyphen and create new columns
application_df[['LOWER_INCOME', 'UPPER_INCOME']] = application_df['INCOME_AMT'].str.split('-', expand=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,LOWER_INCOME,UPPER_INCOME
0,T10,Independent,C1000,ProductDev,Association,Y,0,N,5000,1,0,None
1,T3,Independent,C2000,Preservation,Co-operative,Y,1-9999,N,108590,1,1,9999
2,T5,CompanySponsored,C3000,ProductDev,Association,Y,0,N,5000,0,0,None
3,T3,CompanySponsored,C2000,Preservation,Trust,Y,10000-24999,N,6692,1,10000,24999
4,T3,Independent,C1000,Heathcare,Trust,Y,100000-499999,N,142590,1,100000,499999


In [15]:
# If INCOME_AMT contains "M", convert to e6.
application_df['UPPER_INCOME'] = application_df['UPPER_INCOME'].str.replace('M', 'e6')
application_df['LOWER_INCOME'] = application_df['LOWER_INCOME'].str.replace('M', 'e6')

# Confirm changes
display(application_df[['LOWER_INCOME', 'UPPER_INCOME']].value_counts())
application_df.head()

LOWER_INCOME  UPPER_INCOME
25000         99999           3747
100000        499999          3374
1e6           5e6              955
1             9999             728
10000         24999            543
10e6          50e6             240
5e6           10e6             185
Name: count, dtype: int64

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,LOWER_INCOME,UPPER_INCOME
0,T10,Independent,C1000,ProductDev,Association,Y,0,N,5000,1,0,None
1,T3,Independent,C2000,Preservation,Co-operative,Y,1-9999,N,108590,1,1,9999
2,T5,CompanySponsored,C3000,ProductDev,Association,Y,0,N,5000,0,0,None
3,T3,CompanySponsored,C2000,Preservation,Trust,Y,10000-24999,N,6692,1,10000,24999
4,T3,Independent,C1000,Heathcare,Trust,Y,100000-499999,N,142590,1,100000,499999


In [16]:
# Address 'None' values in UPPER_INCOME. If INCOME_AMT is '0', set UPPER_INCOME to 0.
application_df.loc[application_df['LOWER_INCOME'] == '0', 'UPPER_INCOME'] = 0
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,LOWER_INCOME,UPPER_INCOME
0,T10,Independent,C1000,ProductDev,Association,Y,0,N,5000,1,0,0
1,T3,Independent,C2000,Preservation,Co-operative,Y,1-9999,N,108590,1,1,9999
2,T5,CompanySponsored,C3000,ProductDev,Association,Y,0,N,5000,0,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,Y,10000-24999,N,6692,1,10000,24999
4,T3,Independent,C1000,Heathcare,Trust,Y,100000-499999,N,142590,1,100000,499999


In [17]:
# Address INCOME_AMT values with `50M+`
application_df.loc[application_df['INCOME_AMT'] == '50M+']

# Convert UPPER_INCOME NaN values to 100M
application_df.loc[application_df['INCOME_AMT'] == '50M+', 'UPPER_INCOME'] = 100e6

# Convert to '50M+' to 50e6
application_df['LOWER_INCOME'] = application_df['LOWER_INCOME'].replace('50e6+', '50e6')

# Convert column to integer
application_df['LOWER_INCOME'] = pd.to_numeric(application_df['LOWER_INCOME']).astype(int)
application_df['UPPER_INCOME'] = pd.to_numeric(application_df['UPPER_INCOME']).astype(int)

In [18]:
# Create a new column that compares the ASK_AMT to LOWER_INCOME
application_df['ASK_VS_INCOME'] = application_df['ASK_AMT'] > application_df['LOWER_INCOME']

# Convert boolean to Y/N for consistency
application_df['ASK_VS_INCOME'] = application_df['ASK_VS_INCOME'].map({
    False: 'N',
    True: 'Y'
})

# Display updated dataframe
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,LOWER_INCOME,UPPER_INCOME,ASK_VS_INCOME
0,T10,Independent,C1000,ProductDev,Association,Y,0,N,5000,1,0,0,Y
1,T3,Independent,C2000,Preservation,Co-operative,Y,1-9999,N,108590,1,1,9999,Y
2,T5,CompanySponsored,C3000,ProductDev,Association,Y,0,N,5000,0,0,0,Y
3,T3,CompanySponsored,C2000,Preservation,Trust,Y,10000-24999,N,6692,1,10000,24999,N
4,T3,Independent,C1000,Heathcare,Trust,Y,100000-499999,N,142590,1,100000,499999,Y


### Convert INCOME_AMT column to ordinal

In [19]:
application_df['INCOME_AMT'].value_counts()

INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: count, dtype: int64

In [20]:
# Convert each value to an ordinal score
income_map = {
    '0': 0,
    '1-9999': 1,
    '10000-24999': 2,
    '25000-99999': 3,
    '100000-499999': 4,
    '500000-1M': 5, # No rows fall in this category?
    '1M-5M': 6,
    '5M-10M': 7,
    '10M-50M': 8,
    '50M+': 9
}
application_df['ORDINAL_INCOME_AMT'] = application_df['INCOME_AMT'].map(income_map)

# Drop the INCOME_AMT column
# application_df = application_df.drop(columns=['INCOME_AMT'])

application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,LOWER_INCOME,UPPER_INCOME,ASK_VS_INCOME,ORDINAL_INCOME_AMT
0,T10,Independent,C1000,ProductDev,Association,Y,0,N,5000,1,0,0,Y,0
1,T3,Independent,C2000,Preservation,Co-operative,Y,1-9999,N,108590,1,1,9999,Y,1
2,T5,CompanySponsored,C3000,ProductDev,Association,Y,0,N,5000,0,0,0,Y,0
3,T3,CompanySponsored,C2000,Preservation,Trust,Y,10000-24999,N,6692,1,10000,24999,N,2
4,T3,Independent,C1000,Heathcare,Trust,Y,100000-499999,N,142590,1,100000,499999,Y,4


In [21]:
# Check datatypes before encoding
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  object
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
 10  LOWER_INCOME            34299 non-null  int64 
 11  UPPER_INCOME            34299 non-null  int64 
 12  ASK_VS_INCOME           34299 non-null  object
 13  ORDINAL_INCOME_AMT      34299 non-null  int64 
dtypes: int64(5), object(9)
memory usage: 3.7+ MB


In [40]:
# Drop the INCOME columns
# application_df = application_df.drop(columns=['INCOME_AMT', 'LOWER_INCOME', 'UPPER_INCOME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,ASK_VS_INCOME,ORDINAL_INCOME_AMT
0,T10,Independent,C1000,ProductDev,Association,Y,N,5000,1,Y,0
1,T3,Independent,C2000,Preservation,Co-operative,Y,N,108590,1,Y,1
2,T5,CompanySponsored,C3000,ProductDev,Association,Y,N,5000,0,Y,0
3,T3,CompanySponsored,C2000,Preservation,Trust,Y,N,6692,1,N,2
4,T3,Independent,C1000,Heathcare,Trust,Y,N,142590,1,Y,4


In [41]:
application_df[['AFFILIATION', 'ORGANIZATION']]

,AFFILIATION,ORGANIZATION
0,Independent,Association
1,Independent,Co-operative
2,CompanySponsored,Association
3,CompanySponsored,Trust
4,Independent,Trust
...,...,...
34294,Independent,Association
34295,CompanySponsored,Association
34296,CompanySponsored,Association
34297,Independent,Association


In [44]:
application_df['USE_CASE'].value_counts()

USE_CASE
Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: count, dtype: int64

### Encoding

In [23]:
# Get all the columns which are categorical
categorical_hot = list(application_df.select_dtypes(include="object").columns)

# Convert categorical data to numeric with `pd.get_dummies`
encoded_columns = pd.get_dummies(application_df[categorical_hot]).astype(int)
encoded_columns.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,STATUS_N,STATUS_Y,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_VS_INCOME_N,ASK_VS_INCOME_Y
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,1
1,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,1,1,0,0,1
2,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,1,0,0,1
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,1,0,1,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,1,0,0,1


In [24]:
application_df['IS_SUCCESSFUL'].value_counts()

IS_SUCCESSFUL
1    18261
0    16038
Name: count, dtype: int64

In [25]:
# Split our preprocessed data into our features and target arrays

# Isolate the target array
y = application_df['IS_SUCCESSFUL']

# Isolate the feature array
X = application_df.drop(columns=['IS_SUCCESSFUL'])

# Replace features with encoded equivalents
X.drop(categorical_hot, axis=1, inplace=True)

# Calculate updated feature array
X = pd.concat([X, encoded_columns], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [26]:
# Display the feature array
X.head()

,ASK_AMT,ORDINAL_INCOME_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,...,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,STATUS_N,STATUS_Y,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_VS_INCOME_N,ASK_VS_INCOME_Y
0,5000,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,1
1,108590,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,1,1,0,0,1
2,5000,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,1
3,6692,2,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,1,0,1,0
4,142590,4,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,1,0,0,1


In [27]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [28]:
len(X_train.columns)

66

## Hyperparameter Tuning

In [29]:
number_input_features = len(X_train.columns)
max_hidden_layers = 3 # lowest value permitted = 2, min required for DL
max_num_neurons = number_input_features * 2 - 1 # Was number_input_features * 2 - 1 before
step_count = 5

from tensorflow.keras import layers

def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    
    # Choose activation function in hidden layers
    activation_first_hidden = hp.Choice('activation_layer_0', ['relu', 'leaky_relu', 'tanh', 'elu', 'selu', 'exponential'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(
        units = hp.Int(
            'units_layer_0',
            min_value = 1,
            max_value = max_num_neurons,
            step = step_count),
        activation = activation_first_hidden,
        # kernel_regularizer = tf.keras.regularizers.L1(0.01),
        input_dim = number_input_features
    ))
    
    # Tune whether to use dropout based on the Boolean hyperparameter
    if hp.Boolean("use_dropout"):
        # Add a dropout layer if the Boolean hyperparameter is True
        nn_model.add(layers.Dropout(rate=0.5))  # Adjust the dropout rate as needed
    
    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    num_layers = hp.Int('num_layers', 1, max_hidden_layers-1) # options: 1, 2
    
    for i in range(1, num_layers+1): # i-values: 1, 2 only
        # Choose the number of neurons per layer
        units_layer_i = hp.Int(
            f'units_layer_{i}',
            min_value = 1,
            max_value = max_num_neurons,
            step = step_count
        )
        
        # Choose a different activation function for each layer
        activation_layer_i = hp.Choice(f'activation_layer_{i}', ['relu', 'leaky_relu', 'tanh', 'elu', 'selu', 'exponential'])

        nn_model.add(tf.keras.layers.Dense(
            units = units_layer_i,
            activation = activation_layer_i
        ))

    # Add the output layer
    nn_model.add(tf.keras.layers.Dense(
        units = 1,
        activation = "sigmoid"
    ))

    # Compile the model
    nn_model.compile(
        loss = "binary_crossentropy",
        optimizer = "adam",
        metrics = ["accuracy"]
    )
    
    return(nn_model)

In [30]:
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective = "val_accuracy",
    max_epochs = 20,
    hyperband_iterations = 2
)

In [31]:
tuner.search(
    X_train_scaled,
    y_train,
    epochs = 20,
    validation_data = (X_test_scaled, y_test)
)

Trial 60 Complete [00h 00m 12s]
val_accuracy: 0.7253644466400146

Best val_accuracy So Far: 0.7335277199745178
Total elapsed time: 00h 05m 10s


In [32]:
# Get the top 3 model hyperparameters
best_hyper = tuner.get_best_hyperparameters(3)

from pprint import pprint
for model in best_hyper:
    pprint(model.values)

{'activation_layer_0': 'tanh',
 'activation_layer_1': 'tanh',
 'activation_layer_2': 'exponential',
 'num_layers': 2,
 'tuner/bracket': 2,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/round': 2,
 'tuner/trial_id': '0014',
 'units_layer_0': 56,
 'units_layer_1': 111,
 'units_layer_2': 76,
 'use_dropout': False}
{'activation_layer_0': 'tanh',
 'activation_layer_1': 'relu',
 'activation_layer_2': 'exponential',
 'num_layers': 2,
 'tuner/bracket': 2,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/round': 2,
 'tuner/trial_id': '0042',
 'units_layer_0': 61,
 'units_layer_1': 116,
 'units_layer_2': 101,
 'use_dropout': False}
{'activation_layer_0': 'leaky_relu',
 'activation_layer_1': 'tanh',
 'activation_layer_2': 'tanh',
 'num_layers': 2,
 'tuner/bracket': 2,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/round': 2,
 'tuner/trial_id': '0013',
 'units_layer_0': 116,
 'units_layer_1': 61,
 'units_layer_2': 11,
 'use_dropout': False}


In [ ]:
# Assuming you have a compiled Keras model 'model' and training data 'X_train', 'y_train'
history = nn.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))

# Extract training history
training_loss = history.history['loss']
validation_loss = history.history['val_loss']
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

In [ ]:
from matplotlib import pyplot as plt
# Plotting the learning curves
plt.figure(figsize=(12, 4))

# Plot loss curves
plt.subplot(1, 2, 1)
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.title('Loss Curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot accuracy curves
plt.subplot(1, 2, 2)
plt.plot(training_accuracy, label='Training Accuracy')
plt.plot(validation_accuracy, label='Validation Accuracy')
plt.title('Accuracy Curves')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
'activation_layer_0': 'relu',
 'activation_layer_1': 'relu',
 'activation_layer_2': 'exponential',
 'num_layers': 1,
 'tuner/bracket': 0,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units_layer_0': 181,
 'units_layer_1': 176,
 'units_layer_2': 21

## Compile, Train and Evaluate the Model

In [ ]:
'activation_layer_0': 'relu',
 'activation_layer_1': 'tanh',
 'activation_layer_2': 'relu',
 'num_layers': 2,
 'tuner/bracket': 1,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/round': 1,
 'tuner/trial_id': '0049',
 'units_layer_0': 136,
 'units_layer_1': 141,
 'units_layer_2': 61,
 'use_dropout': False

In [ ]:
'activation_layer_0': 'leaky_relu',
 'activation_layer_1': 'relu',
 'num_layers': 1,
 'tuner/bracket': 2,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/round': 2,
 'tuner/trial_id': '0045',
 'units_layer_0': 81,
 'units_layer_1': 56,
 'use_dropout': False

In [ ]:
'activation_layer_0': 'tanh',
 'activation_layer_1': 'tanh',
 'activation_layer_2': 'exponential',
 'num_layers': 2,
 'tuner/bracket': 2,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/round': 2,
 'tuner/trial_id': '0014',
 'units_layer_0': 56,
 'units_layer_1': 111,
 'units_layer_2': 76,
 'use_dropout': False

In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer0 = 56
hidden_nodes_layer1 = 111
hidden_nodes_layer2 = 76

# Initialise sequential model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(
    units = hidden_nodes_layer0,
    input_dim = number_input_features,
    activation = "tanh"
))

# Second hidden layer
nn.add(tf.keras.layers.Dense(
    units = hidden_nodes_layer1,
    activation = "tanh"
))

# Third hidden layer
nn.add(tf.keras.layers.Dense(
    units = hidden_nodes_layer2,
    activation = "exponential"
))

# # Second to last
# nn.add(tf.keras.layers.Dense(
#     units = 74,
#     activation = "softmax"
# ))

# Output layer
nn.add(tf.keras.layers.Dense(
    units = 1,
    activation = "sigmoid"
))


# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 56)                3752      
                                                                 
 dense_4 (Dense)             (None, 111)               6327      
                                                                 
 dense_5 (Dense)             (None, 76)                8512      
                                                                 
 dense_6 (Dense)             (None, 1)                 77        
                                                                 
Total params: 18668 (72.92 KB)
Trainable params: 18668 (72.92 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
# Compile the model
nn.compile(
    loss = "binary_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy"]
)

In [36]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoints filepath
filepath = "weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5"

# Create a checkpoint
checkpoint = ModelCheckpoint(
    filepath,
    monitor='accuracy',
    verbose=1,
    save_best_only=True,
    mode='max'
)

# Create the callbacks list
callbacks_list = [checkpoint]

In [37]:
# Train the model
fit_model = nn.fit(
    X_train_scaled,
    y_train,
    epochs = 50,
    callbacks = callbacks_list,
    verbose = 0
)


Epoch 1: accuracy improved from -inf to 0.71700, saving model to weights-improvement-01-0.72.hdf5


/Users/alyssahondrade/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: accuracy improved from 0.71700 to 0.72726, saving model to weights-improvement-02-0.73.hdf5

Epoch 3: accuracy improved from 0.72726 to 0.72823, saving model to weights-improvement-03-0.73.hdf5

Epoch 4: accuracy did not improve from 0.72823

Epoch 5: accuracy improved from 0.72823 to 0.72928, saving model to weights-improvement-05-0.73.hdf5

Epoch 6: accuracy did not improve from 0.72928

Epoch 7: accuracy improved from 0.72928 to 0.73107, saving model to weights-improvement-07-0.73.hdf5

Epoch 8: accuracy improved from 0.73107 to 0.73223, saving model to weights-improvement-08-0.73.hdf5

Epoch 9: accuracy improved from 0.73223 to 0.73336, saving model to weights-improvement-09-0.73.hdf5

Epoch 10: accuracy did not improve from 0.73336

Epoch 11: accuracy did not improve from 0.73336

Epoch 12: accuracy did not improve from 0.73336

Epoch 13: accuracy improved from 0.73336 to 0.73394, saving model to weights-improvement-13-0.73.hdf5

Epoch 14: accuracy improved from 0.73394 

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(
    X_test_scaled,
    y_test,
    verbose = 2
)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5790 - accuracy: 0.7270 - 145ms/epoch - 540us/step
Loss: 0.5789932608604431, Accuracy: 0.7269970774650574


In [ ]:
# Export our model to HDF5 file
nn.save('iteration_2.h5', save_format='h5')